# CALVADOS Colab
This Colab notebook enables running molecular dynamics (MD) simulations of intrinsically disordered proteins (IDPs) and multi-domain proteins (MDPs) and to study their conformational ensembles [1].

MD simulations employ the coarse-grained force fields CALVADOS 2 or 3 [2,3], where C$_\alpha$ and center-of-mass (COM) representation will be used for IDPs and MDPs, respectively.

Simulations are run using openMM [4] and require `temperature`, `ionic strength`, `pH`, `sequence` and `domain boundary` (only for MDPs). Detailed instructions can be found in Cell 2.

(Optional) Coarse-grained trajectories are converted to all-atom trajectories using cg2all [5].

Small angle X-ray scattering curves are calculated using polynomial expansions of protein structures and interactions Pepsi-SAXS [6].

Bayesian/Maximum Entropy Reweighting is performed using the BME Python library [7,8].

Usage
MD simulations run on GPU. To enable GPU select Runtime from the menu, then Change runtime type and select GPU.

Note: Cells for preliminary operations should be executed one by one to prevent crashes. This notebook uses condacolab, whose installation will cause a kernel restart. Because of this, a crash will happen during preliminary operations if you execute all cells at once.

### References

If you use this notebook, you may consider citing the following papers:

1. G. Tesei, A. I. Trolle, N. Jonsson, J. Betz, F. E. Knudsen, F. Pesce, K. E. Johansson, K. Lindorff-Larsen __Conformational ensembles of the human intrinsically disordered proteome__ _Nature._ 2024 626:897–904 2023.05.08.539815 DOI: https://doi.org/10.1038/s41586-023-07004-5

2. G. Tesei and K. Lindorff-Larsen __Improved predictions of phase behaviour of intrinsically disordered proteins by tuning the interaction range [version 2; peer review: 2 approved]__ _Open Res Eur._ 2023 2(94) DOI: https://doi.org/10.12688/openreseurope.14967.2

3. Cao, F., von Bülow, S., Tesei, G., & Lindorff‐Larsen, K. (2024). __A coarse‐grained model for disordered and multi‐domain proteins.__ _Protein Science_, 33(11), e5172.

4. P. Eastman, J. Swails, J. D. Chodera et al. __OpenMM 7: Rapid development of high performance algorithms for molecular dynamics__ _PLoS Comput Biol_ 2017 13(7):e1005659 DOI: https://doi.org/10.1371/journal.pcbi.1005659

5. Heo, L., & Feig, M. (2023). __One particle per residue is sufficient to describe all-atom protein structures.__ _bioRxiv_, 2023-05.

6. S. Grudinin, M. Garkavenko, and A. Kazennov __Pepsi-SAXS: an adaptive method for rapid and accurate computation of small-angle x-ray scattering profiles__ _Acta Crystallogr. D Struct. Biol_ 2017 73:449–464 DOI: https://doi.org/10.1107/S2059798317005745

7. S. Bottaro, T. Bengtsen, and K. Lindorff-Larsen __Integrating Molecular Simulation and Experimental Data: A Bayesian/Maximum Entropy Reweighting Approach__ _Methods Mol Biol._ 2020 2112:219–240 DOI: https://doi.org/10.1007/978-1-0716-0270-6_15

8. F. Pesce and K. Lindorff-Larsen __Refining conformational ensembles of flexible proteins against small-angle x-ray scattering data__ _Biophys J._ 2021 120(22):5124–5135 DOI: https://doi.org/10.1016/j.bpj.2021.10.003

In [ ]:
# @title 1. Set the environment for simulation

# @markdown Attention! A prompt of "Your session crashed for an unknown reason" will appear after running this block. This is required for all packages to work properly. Please ignore it and move on.

# @markdown If you have many proteins to run, change the inputs in cell 2, 3 and 4 for each protein as you need. You only need to run cell 1 once.
import subprocess
import locale
locale.getpreferredencoding = lambda: "UTF-8"
subprocess.run('pip install py3Dmol'.split())
subprocess.run('pip install MDAnalysis'.split())
subprocess.run('pip install mdtraj'.split())
subprocess.run('pip install -q condacolab'.split())
import condacolab
condacolab.install()
subprocess.run('mamba install openmm -c conda-forge --yes'.split())
subprocess.run('pip install wget'.split())
subprocess.run('pip install fastprogress'.split())
subprocess.run('mamba install pandas'.split())
subprocess.run('mamba install pydantic'.split())
# !pip install "dgl<=1.0"  # cpu for cg2all

In [ ]:
# @title 2. Configure your protein

#@markdown At https://github.com/KULL-Centre/_2024_Cao_CALVADOSCOM/blob/main/example_data/exp_conditions.csv you can find __temperature__, __ionic strength__, __pH__, __domain boundary__ (for 13 MDPs) and __fasta__ information to reproduce the corresponding SAXS experiment (see DOI: 10.1016/j.bpj.2022.12.013, 10.1002/pro.5172), only for tutorial. You can also find structure files for 13 MDPs in the same _example_data_ directory.

#@markdown Choose the proteins you want to work with and download its “.fasta” file, ".pdb" files (for MDPs) and SAXS data (“.dat” file). Otherwise, you can use your own data.

import numpy as np
import os
import shutil
import ipywidgets as widgets
import warnings
import wget
warnings.filterwarnings('ignore')

example = True  #@param {type:"boolean"}
# @markdown If `example` is checked, all the inputs below in this cell will be ignored. In this case, [THB_C2](https://www.biorxiv.org/content/10.1101/2024.02.03.578735v3) (MDP) will serve as an example for tutorial with pre-defined settings.

isIDP = False #@param {type:"boolean"}
name = "hSUMO_hnRNPA1S" #@param {type:"string"}
sequence = "MGSSHHHHHHGSGLVPRGSASMSDSEVNQEAKPEVKPEVKPETHINLKVSDGSSEIFFKIKKTTPLRRLMEAFAKRQGKEMDSLRFLYDGIRIQADQTPEDLDMEDNDIIEAHREQIGGMSKSESPKEPEQLRKLFIGGLSFETTDESLRSHFEQWGTLTDCVVMRDPNTKRSRGFGFVTYATVEEVDAAMNARPHKVDGRVVEPKRAVSREDSQRPGAHLTVKKIFVGGIKEDTEEHHLRDYFEQYGKIEVIEIMTDRGSGKKRGFAFVTFDDHDSVDKIVIQKYHTVNGHNCEVRKALSKQEMASASSSQRGRSGSGNFGGGRGGGFGGNDNFGRGGNFSGRGGFGGSRGGGGYGGSGDGYNGFGNDGSNFGGGGNYNNQSSNFGPMKGGNFGGRSSGPYGGGGQYFAKPRNQGGYGGSSSSSSYGSGRRF" #@param {type:"string"}
L = int(np.ceil((len(sequence) - 1) * 0.38 + 4))
if L > 900:  # nm
    raise Exception("Too large box! Floats might overflow in PDB file. You can try to modify the 'L' parameter in this block yourself.")
temperature = 293.15 #@param {type:"number"}
ionic_strength = 0.1 #@param {type:"number"}
pH = 7.5 #@param {type:"number"}
cutoff = 2.0  # nm
#@markdown <i>*Units: temperature [K], ionic strength [M]<i>

#@markdown <b>Define domain boundary -- ignore this if `isIDP` is toggled:

#@markdown Residue ranges are delimited by `-`, e.g. residues 1 to 200 are `1-200`;

#@markdown Separate domains are delimited by `,`, e.g. `1-200,201-400`;

#@markdown Discontinuous segments are delimited by `_`, e.g. `1-200,201-300_350-400`.
domain_boundary = "44-114,132-209,224-298" #@param {type:"string"}
input_pae = False
k_restraint = 700  # unit:KJ/(mol*nm^2)
eps_factor = 0.2
initial_type = "C3"
discard_first_nframes = 10

if example:
    name = "THB_C2"
    isIDP = False
    sequence = "GPGSEDVWEILRQAPPSEYERIAFQYGVTDLRGMLKRLKGMRRDEKKSTAFQKKLEPAYQVSKGHKIRLTVELADHDAEVKWLKNGQEIQMSGSKYIFESIGAKRTLTISQCSLADDAAYQCVVGGEKCSTELFVKE"
    L = int(np.ceil((len(sequence) - 1) * 0.38 + 4))
    temperature = 295.15
    ionic_strength = 0.15
    pH = 6.5
    domain_boundary = "6-42,50-137"

use_pdb = False if isIDP else True
use_hnetwork = False if isIDP else True
use_ssdomains = False if isIDP else True

if not isIDP:
    domain_resNum = []  # start from 1
    for domain in domain_boundary.split(","):
        domain = domain.split("_")
        if len(domain) == 1:  # no break point
          domain_resNum.append(list(range(int(domain[0].split('-')[0]), int(domain[0].split('-')[1]) + 1)))
        else:
          tmp_list = []
          for subdomain in domain:
              tmp_list += list(range(int(subdomain.split('-')[0]), int(subdomain.split('-')[1]) + 1))
          domain_resNum.append(tmp_list)
else:
    domain_boundary = None
    domain_resNum = None
    path2pdb = ''

print(f"{'IDP' if isIDP else 'MDP'} name: {name}")
print(f"sequence: {sequence}")
print(f"simulation box: [{L}, {L}, {L}] nm")
print(f"temperature: {temperature} K")
print(f"ionic strength: {ionic_strength} M")
print(f"pH: {pH}")
if not isIDP:
    print(f"There are {len(domain_resNum)} restrained domains (1-based): ")
    for d in domain_resNum:
        print(d)
if not os.path.isdir(f"{name}"):
    os.system(f"mkdir -p {name}")

In [ ]:
# @title 3. Upload a pdb file for MDPs (Skip if your protein is an IDP)

# @markdown *Please make sure the uploaded PDB file starts `resSeq` from 1; Scripts won't check.


from google.colab import files

if not isIDP:
    uploaded = files.upload()
    if len(list(uploaded.keys())) != 1:
      raise Exception("Please upload just one pdb file")
    for key in uploaded.keys():
        os.system(f"mv {key} {name}")
    path2pdb = f"{name}/{list(uploaded.keys())[0]}"
    print(f"{path2pdb} successfully uploaded.")
else:
    print("No need to upload the pdb file")

In [ ]:
# @title 4. Determine simulation time (unit: ns)
import wget
import yaml

#@markdown By checking the "Break_CALVADOS" box, you will add random noise to the amino-acid stickiness parameters of the CALVADOS model ($\lambda$ values). The resulting simulations will not be trustworthy. This function has been added for teaching purposes, so that the effect of reweighting could be appreciated more when using a force field that does not reproduce experimental data accurately.
wget.download('https://github.com/KULL-Centre/_2024_Cao_CALVADOSCOM/raw/main/src/residues_pub.csv')
Break_CALVADOS = False #@param {type:"boolean"}
if Break_CALVADOS == True:
    print ("WARNING: stickiness parameters have been randomly modified. This function is only for teaching purposes.")
os.system(f"mv residues_pub.csv {name}")

#@markdown The default option “AUTO” will set the simulation time depending on sequence length. The longer the IDR, the larger the ensemble of conformations it can adopt. Moreover, the reconfiguration time of IDRs increases with increasing sequence length. Therefore, longer sequences will require more sampling. Typical simulation times range from ca. 5 min (a 71-ns-long simulation of an IDR of 70 residues), 7 min (71-ns-long simulation of an IDR of 140 residues), to 34 min for a 373-ns-long simulation of an IDR of 351 residues.
Simulation_time = "1" #@param {type:"string"}
#@markdown <i>*Units: Simulation_time [ns]<i>
N_res = len(sequence)
if not isIDP:
  domain_len = 0
  for domain in domain_resNum:
    domain_len += len(domain)
  N_res -= domain_len

N_save = 3000 if N_res < 100 else int(np.ceil(3e-4 * N_res ** 2) * 1000)

if Simulation_time == "AUTO":
    N_steps = 210 * int(N_save)
    print(f'AUTO simulation length selected. Running for {N_steps*0.01/1000} ns, {N_steps} steps in total.')
else:
    N_steps = int(float(Simulation_time)*1000/0.01)
    print(f'Simulation length assigned. Running for {Simulation_time} ns, {N_steps} steps in total.')

nframes = N_steps // N_save
config_sim_data = dict(
temp=float(temperature), ionic=float(ionic_strength),
pH=float(pH), cutoff=cutoff,
L=L, wfreq=int(N_save), use_pdb=use_pdb,
path2pdb=path2pdb, use_hnetwork=use_hnetwork, domain_resNum=domain_resNum,
use_ssdomains=use_ssdomains, input_pae=input_pae,
k_restraint=k_restraint, record=name,
gpu_id=0, N_res=int(N_res),
CoarseGrained="COM", isIDP=isIDP,
eps_factor=eps_factor,
initial_type=initial_type, seq=list(sequence), steps=N_steps,
discard_first_nframes=discard_first_nframes,
nframes=nframes, Break_CALVADOS=Break_CALVADOS)
yaml.dump(config_sim_data, open(f'{name}/config_sim.yaml', 'w'))

In [ ]:
# @title 5. Run simulations
import time
from openmm import app
from simtk import openmm, unit
import pandas as pd
import MDAnalysis as mda
import mdtraj as md
from fastprogress import master_bar, progress_bar


def geometry_from_pdb(record, pdb, CoarseGrained="CA", ssdomains=None):
    """ positions in nm """
    u = mda.Universe(pdb)
    ini_CA = u.select_atoms("name CA")
    ini_CA.write(f"{record}/ini_CA.pdb")
    if CoarseGrained == "CA":
        cas = u.select_atoms('name CA')  # in-place
        center_of_mass = cas.center_of_mass()
        cas.translate(-center_of_mass)
        pos = cas.positions / 10.  # nm, shape: (n, 3)
    elif CoarseGrained == "COM":  # COM
        pos = []
        resSeqinDomain = []
        for ssdomain in ssdomains:
            resSeqinDomain += ssdomain
        for i in range(len(u.residues)):  # i starts from 0
            if i+1 in resSeqinDomain:  # COM
                pos.append(u.residues[i].atoms.center_of_mass())
            else:  # CA
                pos.append(np.squeeze(u.residues[i].atoms.select_atoms("name CA").positions))
        center_of_mass = u.atoms.center_of_mass()
        pos = (np.array(pos) - center_of_mass) / 10.  # nm

    return pos, center_of_mass/10.  # nm

def genParamsDH(df,record,prot,temp):
    kT = 8.3145*temp*1e-3
    fasta = prot.fasta.copy()
    r = df.copy()
    q = 1.0

    r.loc['H','q'] = q / ( 1 + 10**(prot.pH-6) )
    r.loc['X'] = r.loc[fasta[0]]
    r.loc['Z'] = r.loc[fasta[-1]]
    fasta[0] = 'X'
    fasta[-1] = 'Z'
    r.loc['X','q'] = r.loc[prot.fasta[0],'q'] + q
    r.loc['Z','q'] = r.loc[prot.fasta[-1],'q'] - q
    # Calculate the prefactor for the Yukawa potential
    fepsw = lambda T : 5321/T+233.76-0.9297*T+0.1417*1e-2*T*T-0.8292*1e-6*T**3
    epsw = fepsw(temp)
    lB = 1.6021766**2/(4*np.pi*8.854188*epsw)*6.022*1000/kT
    yukawa_eps = [r.loc[a].q*np.sqrt(lB*kT) for a in fasta]
    # Calculate the inverse of the Debye length
    yukawa_kappa = np.sqrt(8*np.pi*lB*prot.ionic*6.022/10)
    return yukawa_eps, yukawa_kappa

def genParamsLJ(df,record,prot):
    fasta = prot.fasta.copy()
    r = df.copy()
    r.loc['X'] = r.loc[fasta[0]]
    r.loc['Z'] = r.loc[fasta[-1]]
    r.loc['X','MW'] += 2
    r.loc['Z','MW'] += 16
    fasta[0] = 'X'
    fasta[-1] = 'Z'
    types = list(np.unique(fasta))
    MWs = [r.loc[a,'MW'] for a in types]
    lj_eps = prot.eps_factor*4.184
    return lj_eps, fasta, types, MWs

def load_parameters(record, initial_type, Break_CALVADOS, output=False):
    residues = pd.read_csv(f'{record}/residues_pub.csv').set_index('one', drop=False)
    if initial_type == "C1":
        residues["lambdas"] = residues["CALVADOS1"]
    if initial_type == "C2":
        residues["lambdas"] = residues["CALVADOS2"]
    if initial_type == "C3":
        residues["lambdas"] = residues["CALVADOS3"]
    if Break_CALVADOS:
        np.random.seed(seed=3100)
        residues['lambdas'] += np.random.normal(0,1,residues.index.size)
        residues['lambdas'] = (residues.lambdas-residues.lambdas.min())/(residues.lambdas.max()-residues.lambdas.min())
    residues.to_csv(f'{record}/residues_pub.csv')
    if output:
        print("Lambda used:\n", residues.lambdas)
    return residues

def p2c(r, phi):
    """
    polar to cartesian
    """
    return (r * np.cos(phi), r * np.sin(phi))

def xy_spiral_array(n, delta=0, arc=.38, separation=.7):
    """
    create points on an Archimedes' spiral
    with `arc` giving the length of arc between two points
    and `separation` giving the distance between consecutive
    turnings
    """
    r = arc
    b = separation / (2 * np.pi)
    phi = float(r) / b
    coords = []
    for i in range(n):
        coords.append(list(p2c(r, phi))+[0])
        phi += float(arc) / r
        r = b * phi
    return np.array(coords)+delta

def build_topology(fasta,n_chains=1):
    # build CG topology
    top = md.Topology()
    for i_chain in range(n_chains):
        chain = top.add_chain()
        for resname in fasta:
            residue = top.add_residue(resname, chain)
            top.add_atom(resname, element=md.element.carbon, residue=residue)
        for i in range(chain.n_atoms-1):
            top.add_bond(chain.atom(i),chain.atom(i+1))
    return top

def build_box(Lx,Ly,Lz):
    # set box vectors
    a = unit.Quantity(np.zeros([3]), unit.nanometers)
    a[0] = Lx * unit.nanometers
    b = unit.Quantity(np.zeros([3]), unit.nanometers)
    b[1] = Ly * unit.nanometers
    c = unit.Quantity(np.zeros([3]), unit.nanometers)
    c[2] = Lz * unit.nanometers
    return a, b, c

def add_particles(system, residues, prot, n_chains=1):
    for i_chain in range(n_chains):
        system.addParticle((residues.loc[prot.fasta[0]].MW+2)*unit.amu)
        for a in prot.fasta[1:-1]:
            system.addParticle(residues.loc[a].MW*unit.amu)
        system.addParticle((residues.loc[prot.fasta[-1]].MW+16)*unit.amu)
    return system

def euclidean(a_matrix, b_matrix):
    # Using matrix operation to calculate Euclidean distance
    """                     b
       [[2.23606798  1.          5.47722558]
    a   [7.07106781  4.69041576  3.        ]
        [12.20655562 9.8488578   6.4807407 ]]"""
    d1 = -2 * np.dot(a_matrix, b_matrix.T)
    d2 = np.sum(np.square(a_matrix), axis=1, keepdims=True)
    d3 = np.sum(np.square(b_matrix), axis=1)
    dist = np.sqrt(d1 + d2 + d3)
    return dist

def set_interactions(system, residues, prot, lj_eps, cutoff, yukawa_kappa, yukawa_eps, N, n_chains=1,
                     CoarseGrained="CA", dismatrix=None, isIDP=True, domain_resNum=None):
    hb = openmm.openmm.HarmonicBondForce()
    # interactions
    energy_expression = 'select(step(r-2^(1/6)*s),4*eps*l*((s/r)^12-(s/r)^6-shift),4*eps*((s/r)^12-(s/r)^6-l*shift)+eps*(1-l))'
    ah = openmm.openmm.CustomNonbondedForce(energy_expression + '; s=0.5*(s1+s2); l=0.5*(l1+l2); shift=(0.5*(s1+s2)/rc)^12-(0.5*(s1+s2)/rc)^6')
    ah.addGlobalParameter('eps', lj_eps * unit.kilojoules_per_mole)
    ah.addGlobalParameter('rc', float(cutoff) * unit.nanometer)
    ah.addPerParticleParameter('s')
    ah.addPerParticleParameter('l')
    print('rc', cutoff * unit.nanometer)

    yu = openmm.openmm.CustomNonbondedForce('q*(exp(-kappa*r)/r-shift); q=q1*q2')
    yu.addGlobalParameter('kappa', yukawa_kappa / unit.nanometer)
    yu.addGlobalParameter('shift', np.exp(-yukawa_kappa * 4.0) / 4.0 / unit.nanometer)
    yu.addPerParticleParameter('q')

    for j in range(n_chains):
        begin = j * N  # 0
        end = j * N + N  # n_residues
        for a, e in zip(prot.fasta, yukawa_eps):
            yu.addParticle([e * unit.nanometer * unit.kilojoules_per_mole])
            ah.addParticle([residues.loc[a].sigmas * unit.nanometer, residues.loc[a].lambdas * unit.dimensionless])

        for i in range(begin, end - 1):  # index starts from 0
            if CoarseGrained=="CA" or isIDP:
                hb.addBond(i, i + 1, 0.38 * unit.nanometer, 8033.0 * unit.kilojoules_per_mole / (unit.nanometer ** 2))
            else:  # COM or COE
                ssdomains = domain_resNum
                resSeqinDomain = []
                for ssdomain in ssdomains:
                    resSeqinDomain += ssdomain
                if i+1 in resSeqinDomain or i+1+1 in resSeqinDomain:
                    hb.addBond(i, i + 1, dismatrix[i][i+1] * unit.nanometer, 8033.0 * unit.kilojoules_per_mole / (unit.nanometer ** 2))
                else:
                    hb.addBond(i, i + 1, 0.38 * unit.nanometer, 8033.0 * unit.kilojoules_per_mole / (unit.nanometer ** 2))
            yu.addExclusion(i, i + 1)
            ah.addExclusion(i, i + 1)

    # All Forces in a single force group must use the same cutoff distance
    yu.setForceGroup(0)
    ah.setForceGroup(1)
    yu.setNonbondedMethod(openmm.openmm.CustomNonbondedForce.CutoffPeriodic)
    ah.setNonbondedMethod(openmm.openmm.CustomNonbondedForce.CutoffPeriodic)
    hb.setUsesPeriodicBoundaryConditions(True)
    yu.setCutoffDistance(4 * unit.nanometer)
    ah.setCutoffDistance(cutoff * unit.nanometer)
    return hb, yu, ah

def set_harmonic_network(N,dmap,pae_inv,yu,ah,ssdomains=None,cs_cutoff=0.9,k_restraint=700.):
    cs = openmm.openmm.HarmonicBondForce()
    for i in range(N-2):
        for j in range(i+2,N):
            if ssdomains != None:  # use fixed domain boundaries for network
                ss = False
                for ssdom in ssdomains:
                    if i+1 in ssdom and j+1 in ssdom:
                        ss = True
                if ss:  # both residues in structured domains
                    if dmap[i,j] < cs_cutoff:  # nm
                        cs.addBond(i, j, dmap[i, j] * unit.nanometer,
                                   k_restraint * unit.kilojoules_per_mole / (unit.nanometer ** 2))
                        yu.addExclusion(i, j)
                        ah.addExclusion(i, j)
            elif isinstance(pae_inv, np.ndarray):  # use alphafold PAE matrix for network
                k = k_restraint * pae_inv[i,j]**2
                if k > 0.0:
                    cs.addBond(i,j, dmap[i,j]*unit.nanometer,
                                k*unit.kilojoules_per_mole/(unit.nanometer**2))
                    yu.addExclusion(i, j)
                    ah.addExclusion(i, j)
            else:
                raise
    cs.setUsesPeriodicBoundaryConditions(True)
    return cs, yu, ah

def simulate(config, CUDA_VISIBLE_DEVICES=0):
    """ Simulate openMM Calvados

        * config is a dictionary """

    # parse config
    record, temp, ionic = config['record'], config['temp'], config['ionic']
    cutoff, steps, wfreq = config['cutoff'], config['steps'], config['wfreq']
    L = config['L']
    Break_CALVADOS = config["Break_CALVADOS"]
    domain_resNum = config['domain_resNum']
    eps_factor = config["eps_factor"]
    pH = config["pH"]
    isIDP = config['isIDP']
    CoarseGrained = config['CoarseGrained']
    seq = config['seq']
    replica = 0
    use_pdb, path2pdb = config['use_pdb'], config['path2pdb']
    initial_type = config['initial_type']
    center_of_mass = np.array([0, 0, 0])
    if use_pdb:
        use_hnetwork = config['use_hnetwork']
        if use_hnetwork:
            k_restraint = config['k_restraint']
            use_ssdomains = config['use_ssdomains']
            if use_ssdomains:
                ssdomains = domain_resNum
                pae = None
            else:
                raise Exception("please make sure 'use_ssdomains' is True")
    else:
        path2pdb = ''
        use_hnetwork = False
        pae = None
        ssdomains = None
    # load residue parameters
    residues = load_parameters(record, initial_type, Break_CALVADOS, output=True)
    # build protein dataframe
    df = pd.DataFrame(columns=['pH', 'ionic', 'temp', 'eps_factor', 'fasta'], dtype=object)
    df.loc[record] = dict(pH=pH, ionic=ionic, temp=temp, eps_factor=eps_factor, fasta=seq)
    prot = df.loc[record]
    print("pH:", prot.pH)
    print(f"Ionic strength: {prot.ionic} M")
    print(f"Temperature: {prot.temp} K")
    # LJ and YU parameters
    lj_eps, fasta, types, MWs = genParamsLJ(residues, record, prot)
    # print("lj_eps:", lj_eps * unit.kilojoules_per_mole)
    yukawa_eps, yukawa_kappa = genParamsDH(residues, record, prot, temp)

    N = len(fasta)  # number of residues
    n_chains = 1
    Lz = L

    # get input geometry

    if use_pdb:
        print(f'Starting from pdb structure {path2pdb}')
        pos, center_of_mass = geometry_from_pdb(record, path2pdb, CoarseGrained=CoarseGrained, ssdomains=domain_resNum)
    else:
        spiral = True
        if spiral:
            pos = xy_spiral_array(N)
        else:
            pos = [[L / 2, L / 2, L / 2 + (i - N / 2.) * .38] for i in range(N)]
        pos = np.array(pos)

    top = build_topology(fasta, n_chains=n_chains)
    md.Trajectory(pos + center_of_mass, top, 0, [L, L, Lz], [90, 90, 90]).save_pdb(f'{record}/ini_beads.pdb', force_overwrite=True)
    pos = pos + np.array([L / 2, L / 2, L / 2])
    a = md.Trajectory(pos, top, 0, [L, L, Lz], [90, 90, 90])
    a.save_pdb(f'{record}/top_production.pdb', force_overwrite=True)
    # build openmm system
    system = openmm.System()

    # box
    a, b, c = build_box(L, L, Lz)
    system.setDefaultPeriodicBoxVectors(a, b, c)

    # load topology into system
    pdb = app.pdbfile.PDBFile(f'{record}/top_production.pdb')

    # print(pdb.topology)

    # particles and termini
    system = add_particles(system, residues, prot, n_chains=n_chains)
    dmap = euclidean(pos, pos)
    # interactions
    hb, yu, ah = set_interactions(system, residues, prot, lj_eps, cutoff, yukawa_kappa, yukawa_eps, N,
                                  n_chains=n_chains, CoarseGrained=CoarseGrained, dismatrix=dmap, isIDP=isIDP,
                                  domain_resNum=domain_resNum)
    system.addForce(hb)
    system.addForce(yu)
    system.addForce(ah)

    # print("use_hnetwork: ", use_hnetwork)
    # harmonic network (currently unavailable for slab sim)
    if use_hnetwork:
        cs, yu, ah = set_harmonic_network(N, dmap, pae, yu, ah, ssdomains=ssdomains, k_restraint=k_restraint)
        # print(f"k_restraint used: {k_restraint}")
        system.addForce(cs)

    open(f'{record}/system.xml', 'w').write(openmm.XmlSerializer.serialize(system))
    # use langevin integrator
    integrator = openmm.openmm.LangevinIntegrator(temp * unit.kelvin, 0.01 / unit.picosecond, 0.01 * unit.picosecond)
    # print(integrator.getFriction(), integrator.getTemperature())

    try:
        os.environ["CUDA_VISIBLE_DEVICES"] = f"{CUDA_VISIBLE_DEVICES}"  # feasible
        os.system("echo $CUDA_VISIBLE_DEVICES")
        simulation = app.simulation.Simulation(pdb.topology, system, integrator,
                                               openmm.Platform.getPlatformByName("CUDA")
                                               ,{"DeviceIndex": f"{CUDA_VISIBLE_DEVICES}"})
        print("Using GPU acceleration")
    except Exception:
        platform = openmm.Platform.getPlatformByName('CPU')
        simulation = app.simulation.Simulation(pdb.topology, system, integrator, platform, dict(Threads=str(1)))
        print("Using CPU")

    simulation.context.setPositions(pdb.positions)
    simulation.minimizeEnergy()
    simulation.reporters.append(app.dcdreporter.DCDReporter(f'{record}/production.dcd', wfreq, enforcePeriodicBox=False, append=False))

    simulation.reporters.append(
        app.statedatareporter.StateDataReporter(f'{record}/statedata_production.log', int(wfreq),
                                                step=True, speed=True, elapsedTime=True, separator='\t', progress=True,
                                                remainingTime=True, totalSteps=steps))
    simulation.reporters.append(
        app.checkpointreporter.CheckpointReporter(file=f"{record}/checkpoint_production.chk",
                                                  reportInterval=wfreq))


    starttime = time.time()  # begin timer
    # assert steps%wfreq==0
    print(f"Total frames: {steps//wfreq}; the first 10 frames will be discarded")
    for _ in progress_bar(list(range(steps//wfreq))):
        simulation.step(wfreq)
    endtime = time.time()  # end timer
    target_seconds = endtime - starttime  # total used time
    print(
        f"{record} total simulations used time: {target_seconds // 3600}h {(target_seconds // 60) % 60}min {np.round(target_seconds % 60, 2)}s")

def fix_topology(dcd,pdb):
    """
    Changes atom names to CA; even if CG-strategy is COM, it is still CA for convenience
    """
    t = md.load_dcd(dcd,pdb)
    cgtop = md.Topology()
    cgchain = cgtop.add_chain()
    for atom in t.top.atoms:
        cgres = cgtop.add_residue(atom.name, cgchain)
        cgtop.add_atom('CA', element=md.element.carbon, residue=cgres)
    traj = md.Trajectory(t.xyz, cgtop, t.time, t.unitcell_lengths, t.unitcell_angles)
    traj = traj.superpose(traj, frame=0)
    return traj

def visualize_traj(record):
    traj_dcd = fix_topology(f'{record}/{record}_cg.dcd', f'{record}/{record}_cg.pdb')
    traj_dcd = traj_dcd.superpose(traj_dcd[0])
    traj_dcd[0].save_pdb(f'{record}/{record}_first_cg.pdb')
    traj_dcd[-1].save_pdb(f'{record}/{record}_last_cg.pdb')
    traj_dcd.save_trr(f"{record}/{record}_cg.trr")
    traj_dcd.save_dcd(f"{record}/{record}_CA.dcd")

def centerDCD(config):
    record = config["record"]
    discard_first_nframes = config["discard_first_nframes"]
    initial_type = config["initial_type"]
    seq = config["seq"]
    Break_CALVADOS = config["Break_CALVADOS"]
    residues = load_parameters(record, initial_type, Break_CALVADOS)
    top = md.Topology()
    chain = top.add_chain()
    for resname in seq:
        residue = top.add_residue(residues.loc[resname, 'three'], chain)
        top.add_atom(residues.loc[resname, 'three'], element=md.element.carbon, residue=residue)
    for i in range(len(seq) - 1):
        top.add_bond(top.atom(i), top.atom(i + 1))
    traj = md.load_dcd(f"{record}/production.dcd", top=top)[discard_first_nframes:]
    traj = traj.image_molecules(inplace=False, anchor_molecules=[set(traj.top.chain(0).atoms)], make_whole=True)
    traj.center_coordinates()
    traj.xyz += traj.unitcell_lengths[0, 0] / 2
    # print(f'Number of frames: {traj.n_frames}')
    traj.save_dcd(f'{record}/{record}_cg.dcd')
    traj[0].save_pdb(f'{record}/{record}_cg.pdb')
    visualize_traj(record)

def calRg(record, initial_type, Break_CALVADOS):
    df = load_parameters(record, initial_type, Break_CALVADOS).set_index("three")
    t = md.load_dcd(f"{record}/{record}_cg.dcd", f"{record}/{record}_cg.pdb")  # nm
    residues = [res.name for res in t.top.atoms]
    masses = df.loc[residues,'MW'].values
    masses[0] += 2
    masses[-1] += 16
    # calculate the center of mass
    cm = np.sum(t.xyz*masses[np.newaxis,:,np.newaxis],axis=1)/masses.sum()
    # calculate residue-cm distances
    si = np.linalg.norm(t.xyz - cm[:,np.newaxis,:],axis=2)
    # calculate rg
    rgarray = np.sqrt(np.sum(si**2*masses,axis=1)/masses.sum())
    np.save(f"{record}/Rg_traj.npy", rgarray)  # this contains Rg trajectory (n_frames,)
    print(f"Rg: {np.mean(rgarray)} nm")


config = yaml.safe_load(open(f"{name}/config_sim.yaml", 'r'))
simulate(config, "0")
centerDCD(config)

In [ ]:
# @title 6. Analysis
calRg(config["record"], config["initial_type"], config["Break_CALVADOS"])

In [ ]:
# @title Visualizing trajectory
import py3Dmol
class Atom(dict):
    def __init__(self, line):
        self["type"] = line[0:6].strip()
        self["idx"] = line[6:11].strip()
        self["name"] = line[12:16].strip()
        self["resname"] = line[17:20].strip()
        self["resid"] = int(int(line[22:26]))
        self["x"] = float(line[30:38])
        self["y"] = float(line[38:46])
        self["z"] = float(line[46:54])
        self["sym"] = line[76:78].strip()
    def __str__(self):
        line = list(" " * 80)

        line[0:6] = self["type"].ljust(6)
        line[6:11] = self["idx"].ljust(5)
        line[12:16] = self["name"].ljust(4)
        line[17:20] = self["resname"].ljust(3)
        line[22:26] = str(self["resid"]).ljust(4)
        line[30:38] = str(self["x"]).rjust(8)
        line[38:46] = str(self["y"]).rjust(8)
        line[46:54] = str(self["z"]).rjust(8)
        line[76:78] = self["sym"].rjust(2)
        return "".join(line) + "\n"

class Molecule(list):
    def __init__(self, file):
        for line in file:
            if "ATOM" in line or "HETATM" in line:
                self.append(Atom(line))

    def __str__(self):
        outstr = ""
        for at in self:
            outstr += str(at)

        return outstr

if not config["isIDP"]:
    all_domain_residues = []
    for temp in domain_resNum:
        all_domain_residues += temp
    template = Molecule(open(f'{config["record"]}/{config["record"]}_cg.pdb', 'r'))
    for at in template:
        if int(at["idx"]) in all_domain_residues:
            at["pymol"] = {"sphere": {'color': "red"}}

md.load_dcd(f'{config["record"]}/{config["record"]}_cg.dcd', f'{config["record"]}/{config["record"]}_cg.pdb').save_pdb(f'{config["record"]}/{config["record"]}_view.pdb')
traj = md.load_pdb(f'{config["record"]}/{config["record"]}_view.pdb')
domain_resNum = config["domain_resNum"]
traj = traj.superpose(traj[0], atom_indices=np.array(domain_resNum[0]))
traj.save_pdb(f'{config["record"]}/{config["record"]}_view.pdb')
with open(f'{config["record"]}/{config["record"]}_view.pdb') as ifile:
    view = py3Dmol.view(width=400, height=300)
    view.addModelsAsFrames("".join([x for x in ifile]))
    if not config["isIDP"]:
        for i, at in enumerate(template):
            default = {"sphere": {'color': 'cyan'}}
            view.setStyle({'model': -1, 'serial': i+1}, at.get("pymol", default))
    else:
        view.setStyle({'model': -1}, {"sphere": {'color': 'cyan'}})
    view.zoomTo()
    view.animate({'loop': "forward"})
    view.show()

In [ ]:
# @title 7. Download results

# @markdown In this zip file:

# @markdown `${protein_name}_cg.dcd`: the coarse-grained trajectory file;

# @markdown `${protein_name}_cg.pdb`: the coarse-grained topology file;

# @markdown After reconstruction (Optional)

# @markdown `$protein_name_AA.dcd`: the reconstructed trajectory file;

# @markdown `$protein_name_topAA.pdb`: the all-atom topology file;
import subprocess
zipper = f'zip -r {name}.zip {name}'
subprocess.run(zipper.split())
files.download(f'{name}.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# @title (Optional) 8.1 Install [cg2all](https://www.sciencedirect.com/science/article/pii/S096921262300401X?via%3Dihub) for all-atom reconstruction (5 mins)
%%bash
pip install dgl -f https://data.dgl.ai/wheels/torch-2.3/cu121/repo.html &> /dev/null
pip install -q git+http://github.com/huhlim/cg2all@cuda-12 &> /dev/null
pip install -q py3Dmol gdown mrcfile &> /dev/null

In [ ]:
#@title (Optional) 8.2 Reconstruct all-atom trajectory
import locale
locale.getpreferredencoding = lambda: "UTF-8"
import yaml
config = yaml.safe_load(open(f"{name}/config_sim.yaml", 'r'))
record = config["record"]
isIDP = config["isIDP"]
cg_model = "CalphaBasedModel" if isIDP else "ResidueBasedModel"
print(f"using {cg_model}")
!convert_cg2all -p {record}/{record}_first_cg.pdb -d {record}/{record}_cg.dcd -o {record}/{record}_AA.dcd -opdb {record}/{record}_topAA.pdb --cg {cg_model}